# ⚡ เปรียบเทียบประสิทธิภาพ: Optimization Algorithms

**จุดประสงค์**: เปรียบเทียบ **ความเร็ว, ความแม่นยำ, และการ converge** ของ Optimizers ต่างๆ

**หมวดหมู่**: `machine_learning_examples-master/ann_class2`

---

## 🎯 Optimizers ที่จะเปรียบเทียบ:

| # | Optimizer | ไฟล์ต้นฉบับ | จุดเด่น |
|---|-----------|------------|--------|
| 1 | **SGD** | `sgd.py` | ง่ายที่สุด, baseline |
| 2 | **Momentum** | `momentum.py` | เร็วกว่า SGD, ลดการกระเด้ง |
| 3 | **RMSprop** | `rmsprop.py` | Adaptive learning rate |
| 4 | **Adam** | `adam.py` | รวมข้อดีของ Momentum + RMSprop |

---

## 📊 เกณฑ์การเปรียบเทียบ:

### 1️⃣ **Convergence Speed** (ความเร็วในการลู่เข้า)
- Loss ลดลงเร็วแค่ไหน?
- Accuracy เพิ่มขึ้นเร็วแค่ไหน?

### 2️⃣ **Final Performance** (ประสิทธิภาพสุดท้าย)
- Final Loss ต่ำสุดที่ได้
- Final Accuracy สูงสุดที่ได้

### 3️⃣ **Stability** (ความเสถียร)
- Loss curve เรียบหรือกระเด้ง?
- มี oscillations มากแค่ไหน?

### 4️⃣ **Hyperparameter Sensitivity** (ความไวต่อ hyperparameters)
- ต้องตั้งค่ายาก-ง่ายแค่ไหน?
- ทนทานต่อการเลือก learning rate แค่ไหน?

---

## 🧪 ปัญหาทดสอบ:

**MNIST Digit Recognition** (เหมือนใน adam.py, momentum.py):
- Input: 784 pixels (28×28 images)
- Output: 10 classes (digits 0-9)
- Data: 60,000 training + 10,000 test samples
- Architecture: **784 → 300 → 100 → 10**

---

## 🔬 วิธีการทดสอบ:

1. ใช้ **architecture เดียวกัน** สำหรับทุก optimizer
2. ใช้ **learning rate เดียวกัน** (0.001) สำหรับความยุติธรรม
3. ใช้ **batch size เดียวกัน** (100)
4. Train **epochs เท่ากัน** (20 epochs)
5. เปรียบเทียบ loss curves และ accuracy curves

## 📦 Step 1: Setup & Load Data

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import pandas as pd
import sys
from sklearn.utils import shuffle
import warnings
warnings.filterwarnings('ignore')

# ตั้งค่าฟอนต์ไทย
font_path = '../../font/Prompt/Prompt-Regular.ttf'
font_prop = fm.FontProperties(fname=font_path)
fm.fontManager.addfont(font_path)
plt.rcParams['font.family'] = font_prop.get_name()
plt.rcParams['axes.unicode_minus'] = False

# เพิ่ม path
sys.path.append('../../machine_learning_examples-master')

# Import utility functions
from ann_class2.util import get_normalized_data, y2indicator

print("✅ Setup เรียบร้อย!")

In [ ]:
# Load MNIST data
print("📥 Loading MNIST dataset...\n")

X, Y = get_normalized_data()

# Split data
X_train = X[:-1000]
Y_train = Y[:-1000]
X_test = X[-1000:]
Y_test = Y[-1000:]

# One-hot encode
Y_train_ind = y2indicator(Y_train)
Y_test_ind = y2indicator(Y_test)

print(f"✅ Data loaded successfully!")
print(f"   • Training samples: {len(X_train)}")
print(f"   • Test samples: {len(X_test)}")
print(f"   • Input features: {X_train.shape[1]}")
print(f"   • Output classes: {Y_train_ind.shape[1]}")

## 🏗️ Step 2: Build Neural Network Class

**Architecture**: 784 → 300 → 100 → 10

In [ ]:
class NeuralNetwork:
    """
    Neural Network with customizable optimizer
    
    Architecture: 784 → 300 → 100 → 10
    Activation: ReLU (hidden), Softmax (output)
    """
    
    def __init__(self, M1=300, M2=100):
        self.M1 = M1  # Hidden layer 1 size
        self.M2 = M2  # Hidden layer 2 size
        
    def fit(self, X, Y, Xtest, Ytest, optimizer='adam', 
            learning_rate=0.001, epochs=20, batch_size=100, 
            beta1=0.9, beta2=0.999, eps=1e-8, mu=0.9, decay_rate=0.999):
        """
        Train the network
        
        Parameters:
        - optimizer: 'sgd', 'momentum', 'rmsprop', 'adam'
        - beta1: momentum parameter for Adam (default: 0.9)
        - beta2: RMSprop parameter for Adam (default: 0.999)
        - mu: momentum parameter for Momentum optimizer (default: 0.9)
        - decay_rate: decay rate for RMSprop (default: 0.999)
        """
        # Initialize
        N, D = X.shape
        K = Y.shape[1]
        self.N = N
        self.K = K
        
        # Initialize weights
        self.W1 = np.random.randn(D, self.M1) / np.sqrt(D)
        self.b1 = np.zeros(self.M1)
        self.W2 = np.random.randn(self.M1, self.M2) / np.sqrt(self.M1)
        self.b2 = np.zeros(self.M2)
        self.W3 = np.random.randn(self.M2, K) / np.sqrt(self.M2)
        self.b3 = np.zeros(K)
        
        # Initialize optimizer-specific variables
        if optimizer == 'momentum':
            # Momentum velocities
            self.vW1 = 0
            self.vb1 = 0
            self.vW2 = 0
            self.vb2 = 0
            self.vW3 = 0
            self.vb3 = 0
            
        elif optimizer == 'rmsprop':
            # RMSprop cache
            self.cW1 = 1
            self.cb1 = 1
            self.cW2 = 1
            self.cb2 = 1
            self.cW3 = 1
            self.cb3 = 1
            
        elif optimizer == 'adam':
            # Adam: momentum + RMSprop
            self.mW1 = 0
            self.mb1 = 0
            self.mW2 = 0
            self.mb2 = 0
            self.mW3 = 0
            self.mb3 = 0
            self.vW1 = 0
            self.vb1 = 0
            self.vW2 = 0
            self.vb2 = 0
            self.vW3 = 0
            self.vb3 = 0
        
        # Track metrics
        self.train_costs = []
        self.test_costs = []
        self.train_accuracies = []
        self.test_accuracies = []
        
        # Training
        n_batches = N // batch_size
        
        for epoch in range(epochs):
            # Shuffle data
            X_shuffled, Y_shuffled = shuffle(X, Y)
            
            for batch_idx in range(n_batches):
                # Get batch
                start = batch_idx * batch_size
                end = start + batch_size
                X_batch = X_shuffled[start:end]
                Y_batch = Y_shuffled[start:end]
                
                # Forward pass
                pY, Z1, Z2 = self.forward(X_batch)
                
                # Gradients
                pY_Y = pY - Y_batch
                
                # Backpropagation
                self.gW3 = Z2.T.dot(pY_Y)
                self.gb3 = pY_Y.sum(axis=0)
                
                dZ2 = pY_Y.dot(self.W3.T) * (Z2 > 0)  # ReLU derivative
                self.gW2 = Z1.T.dot(dZ2)
                self.gb2 = dZ2.sum(axis=0)
                
                dZ1 = dZ2.dot(self.W2.T) * (Z1 > 0)  # ReLU derivative
                self.gW1 = X_batch.T.dot(dZ1)
                self.gb1 = dZ1.sum(axis=0)
                
                # Update weights based on optimizer
                if optimizer == 'sgd':
                    self.update_sgd(learning_rate)
                elif optimizer == 'momentum':
                    self.update_momentum(learning_rate, mu)
                elif optimizer == 'rmsprop':
                    self.update_rmsprop(learning_rate, decay_rate, eps)
                elif optimizer == 'adam':
                    self.update_adam(learning_rate, beta1, beta2, eps, epoch * n_batches + batch_idx + 1)
            
            # Calculate costs
            pY_train = self.predict(X)
            train_cost = self.cost(Y, pY_train)
            train_acc = self.classification_rate(np.argmax(Y, axis=1), np.argmax(pY_train, axis=1))
            
            pY_test = self.predict(Xtest)
            test_cost = self.cost(Ytest, pY_test)
            test_acc = self.classification_rate(np.argmax(Ytest, axis=1), np.argmax(pY_test, axis=1))
            
            self.train_costs.append(train_cost)
            self.test_costs.append(test_cost)
            self.train_accuracies.append(train_acc)
            self.test_accuracies.append(test_acc)
            
            if epoch % 5 == 0:
                print(f"Epoch {epoch:2d} | Train: {train_cost:.4f}, {train_acc:.4f} | Test: {test_cost:.4f}, {test_acc:.4f}")
    
    def forward(self, X):
        """Forward propagation"""
        Z1 = np.maximum(0, X.dot(self.W1) + self.b1)  # ReLU
        Z2 = np.maximum(0, Z1.dot(self.W2) + self.b2)  # ReLU
        Y = self.softmax(Z2.dot(self.W3) + self.b3)
        return Y, Z1, Z2
    
    def predict(self, X):
        """Predict probabilities"""
        Y, _, _ = self.forward(X)
        return Y
    
    def softmax(self, A):
        """Softmax activation"""
        expA = np.exp(A - np.max(A, axis=1, keepdims=True))  # numerical stability
        return expA / expA.sum(axis=1, keepdims=True)
    
    def cost(self, T, Y):
        """Cross-entropy loss"""
        return -(T * np.log(Y + 1e-10)).sum()
    
    def classification_rate(self, Y, P):
        """Classification accuracy"""
        return np.mean(Y == P)
    
    # ===== Optimizer Update Rules =====
    
    def update_sgd(self, lr):
        """Stochastic Gradient Descent"""
        self.W3 -= lr * self.gW3
        self.b3 -= lr * self.gb3
        self.W2 -= lr * self.gW2
        self.b2 -= lr * self.gb2
        self.W1 -= lr * self.gW1
        self.b1 -= lr * self.gb1
    
    def update_momentum(self, lr, mu):
        """Momentum (SGD with velocity)"""
        # Update velocities
        self.vW3 = mu * self.vW3 - lr * self.gW3
        self.vb3 = mu * self.vb3 - lr * self.gb3
        self.vW2 = mu * self.vW2 - lr * self.gW2
        self.vb2 = mu * self.vb2 - lr * self.gb2
        self.vW1 = mu * self.vW1 - lr * self.gW1
        self.vb1 = mu * self.vb1 - lr * self.gb1
        
        # Update weights
        self.W3 += self.vW3
        self.b3 += self.vb3
        self.W2 += self.vW2
        self.b2 += self.vb2
        self.W1 += self.vW1
        self.b1 += self.vb1
    
    def update_rmsprop(self, lr, decay, eps):
        """RMSprop (adaptive learning rate)"""
        # Update cache (moving average of squared gradients)
        self.cW3 = decay * self.cW3 + (1 - decay) * self.gW3 * self.gW3
        self.cb3 = decay * self.cb3 + (1 - decay) * self.gb3 * self.gb3
        self.cW2 = decay * self.cW2 + (1 - decay) * self.gW2 * self.gW2
        self.cb2 = decay * self.cb2 + (1 - decay) * self.gb2 * self.gb2
        self.cW1 = decay * self.cW1 + (1 - decay) * self.gW1 * self.gW1
        self.cb1 = decay * self.cb1 + (1 - decay) * self.gb1 * self.gb1
        
        # Update weights (adaptive learning rate)
        self.W3 -= lr * self.gW3 / (np.sqrt(self.cW3) + eps)
        self.b3 -= lr * self.gb3 / (np.sqrt(self.cb3) + eps)
        self.W2 -= lr * self.gW2 / (np.sqrt(self.cW2) + eps)
        self.b2 -= lr * self.gb2 / (np.sqrt(self.cb2) + eps)
        self.W1 -= lr * self.gW1 / (np.sqrt(self.cW1) + eps)
        self.b1 -= lr * self.gb1 / (np.sqrt(self.cb1) + eps)
    
    def update_adam(self, lr, beta1, beta2, eps, t):
        """Adam (Momentum + RMSprop with bias correction)"""
        # Update biased first moment (momentum)
        self.mW3 = beta1 * self.mW3 + (1 - beta1) * self.gW3
        self.mb3 = beta1 * self.mb3 + (1 - beta1) * self.gb3
        self.mW2 = beta1 * self.mW2 + (1 - beta1) * self.gW2
        self.mb2 = beta1 * self.mb2 + (1 - beta1) * self.gb2
        self.mW1 = beta1 * self.mW1 + (1 - beta1) * self.gW1
        self.mb1 = beta1 * self.mb1 + (1 - beta1) * self.gb1
        
        # Update biased second moment (RMSprop)
        self.vW3 = beta2 * self.vW3 + (1 - beta2) * self.gW3 * self.gW3
        self.vb3 = beta2 * self.vb3 + (1 - beta2) * self.gb3 * self.gb3
        self.vW2 = beta2 * self.vW2 + (1 - beta2) * self.gW2 * self.gW2
        self.vb2 = beta2 * self.vb2 + (1 - beta2) * self.gb2 * self.gb2
        self.vW1 = beta2 * self.vW1 + (1 - beta2) * self.gW1 * self.gW1
        self.vb1 = beta2 * self.vb1 + (1 - beta2) * self.gb1 * self.gb1
        
        # Bias correction
        mW3_hat = self.mW3 / (1 - beta1**t)
        mb3_hat = self.mb3 / (1 - beta1**t)
        mW2_hat = self.mW2 / (1 - beta1**t)
        mb2_hat = self.mb2 / (1 - beta1**t)
        mW1_hat = self.mW1 / (1 - beta1**t)
        mb1_hat = self.mb1 / (1 - beta1**t)
        
        vW3_hat = self.vW3 / (1 - beta2**t)
        vb3_hat = self.vb3 / (1 - beta2**t)
        vW2_hat = self.vW2 / (1 - beta2**t)
        vb2_hat = self.vb2 / (1 - beta2**t)
        vW1_hat = self.vW1 / (1 - beta2**t)
        vb1_hat = self.vb1 / (1 - beta2**t)
        
        # Update weights
        self.W3 -= lr * mW3_hat / (np.sqrt(vW3_hat) + eps)
        self.b3 -= lr * mb3_hat / (np.sqrt(vb3_hat) + eps)
        self.W2 -= lr * mW2_hat / (np.sqrt(vW2_hat) + eps)
        self.b2 -= lr * mb2_hat / (np.sqrt(vb2_hat) + eps)
        self.W1 -= lr * mW1_hat / (np.sqrt(vW1_hat) + eps)
        self.b1 -= lr * mb1_hat / (np.sqrt(vb1_hat) + eps)

print("✅ Neural Network class พร้อมแล้ว!")

## 🏋️ Step 3: Train with Different Optimizers

### 1️⃣ SGD (Stochastic Gradient Descent)

In [ ]:
print("🏃 Training with SGD (Baseline)...\n")

model_sgd = NeuralNetwork()
model_sgd.fit(
    X_train, Y_train_ind, X_test, Y_test_ind,
    optimizer='sgd',
    learning_rate=0.001,
    epochs=20,
    batch_size=100
)

print(f"\n✅ SGD Training Complete!")
print(f"   • Final Train Accuracy: {model_sgd.train_accuracies[-1]:.4f} ({model_sgd.train_accuracies[-1]*100:.1f}%)")
print(f"   • Final Test Accuracy: {model_sgd.test_accuracies[-1]:.4f} ({model_sgd.test_accuracies[-1]*100:.1f}%)")

### 2️⃣ Momentum

In [ ]:
print("🚀 Training with Momentum...\n")

model_momentum = NeuralNetwork()
model_momentum.fit(
    X_train, Y_train_ind, X_test, Y_test_ind,
    optimizer='momentum',
    learning_rate=0.001,
    epochs=20,
    batch_size=100,
    mu=0.9  # momentum parameter
)

print(f"\n✅ Momentum Training Complete!")
print(f"   • Final Train Accuracy: {model_momentum.train_accuracies[-1]:.4f} ({model_momentum.train_accuracies[-1]*100:.1f}%)")
print(f"   • Final Test Accuracy: {model_momentum.test_accuracies[-1]:.4f} ({model_momentum.test_accuracies[-1]*100:.1f}%)")

### 3️⃣ RMSprop

In [ ]:
print("📊 Training with RMSprop...\n")

model_rmsprop = NeuralNetwork()
model_rmsprop.fit(
    X_train, Y_train_ind, X_test, Y_test_ind,
    optimizer='rmsprop',
    learning_rate=0.001,
    epochs=20,
    batch_size=100,
    decay_rate=0.999  # RMSprop decay
)

print(f"\n✅ RMSprop Training Complete!")
print(f"   • Final Train Accuracy: {model_rmsprop.train_accuracies[-1]:.4f} ({model_rmsprop.train_accuracies[-1]*100:.1f}%)")
print(f"   • Final Test Accuracy: {model_rmsprop.test_accuracies[-1]:.4f} ({model_rmsprop.test_accuracies[-1]*100:.1f}%)")

### 4️⃣ Adam (Adaptive Moment Estimation)

In [ ]:
print("⚡ Training with Adam...\n")

model_adam = NeuralNetwork()
model_adam.fit(
    X_train, Y_train_ind, X_test, Y_test_ind,
    optimizer='adam',
    learning_rate=0.001,
    epochs=20,
    batch_size=100,
    beta1=0.9,    # momentum
    beta2=0.999   # RMSprop
)

print(f"\n✅ Adam Training Complete!")
print(f"   • Final Train Accuracy: {model_adam.train_accuracies[-1]:.4f} ({model_adam.train_accuracies[-1]*100:.1f}%)")
print(f"   • Final Test Accuracy: {model_adam.test_accuracies[-1]:.4f} ({model_adam.test_accuracies[-1]*100:.1f}%)")

## 📊 Step 4: สรุปเปรียบเทียบทั้งหมด

### 🏆 Performance Comparison Table

In [ ]:
# สร้างตารางเปรียบเทียบ
comparison_data = {
    'Optimizer': ['SGD', 'Momentum', 'RMSprop', 'Adam'],
    'Train Accuracy': [
        f"{model_sgd.train_accuracies[-1]:.2%}",
        f"{model_momentum.train_accuracies[-1]:.2%}",
        f"{model_rmsprop.train_accuracies[-1]:.2%}",
        f"{model_adam.train_accuracies[-1]:.2%}"
    ],
    'Test Accuracy': [
        f"{model_sgd.test_accuracies[-1]:.2%}",
        f"{model_momentum.test_accuracies[-1]:.2%}",
        f"{model_rmsprop.test_accuracies[-1]:.2%}",
        f"{model_adam.test_accuracies[-1]:.2%}"
    ],
    'Final Train Loss': [
        f"{model_sgd.train_costs[-1]:.2f}",
        f"{model_momentum.train_costs[-1]:.2f}",
        f"{model_rmsprop.train_costs[-1]:.2f}",
        f"{model_adam.train_costs[-1]:.2f}"
    ],
    'Converge Speed': ['⭐', '⭐⭐⭐', '⭐⭐⭐⭐', '⭐⭐⭐⭐⭐'],
    'Stability': ['⭐⭐', '⭐⭐⭐', '⭐⭐⭐⭐', '⭐⭐⭐⭐⭐'],
    'Ease of Tuning': ['⭐⭐', '⭐⭐⭐', '⭐⭐⭐⭐', '⭐⭐⭐⭐⭐']
}

df_comparison = pd.DataFrame(comparison_data)

print("\n" + "="*80)
print("🏆 PERFORMANCE COMPARISON: Optimization Algorithms")
print("="*80 + "\n")
print(df_comparison.to_string(index=False))
print("\n" + "="*80)

### 📈 Visualize Learning Curves

In [ ]:
# Plot Loss Curves
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Plot 1: Train Loss
axes[0].plot(model_sgd.train_costs, label='SGD', linewidth=2, alpha=0.8)
axes[0].plot(model_momentum.train_costs, label='Momentum', linewidth=2, alpha=0.8)
axes[0].plot(model_rmsprop.train_costs, label='RMSprop', linewidth=2, alpha=0.8)
axes[0].plot(model_adam.train_costs, label='Adam', linewidth=2, alpha=0.8)
axes[0].set_title('📉 Training Loss', fontsize=13, fontweight='bold')
axes[0].set_xlabel('Epoch', fontsize=11)
axes[0].set_ylabel('Loss', fontsize=11)
axes[0].legend(fontsize=10)
axes[0].grid(alpha=0.3, linestyle='--')

# Plot 2: Test Loss
axes[1].plot(model_sgd.test_costs, label='SGD', linewidth=2, alpha=0.8)
axes[1].plot(model_momentum.test_costs, label='Momentum', linewidth=2, alpha=0.8)
axes[1].plot(model_rmsprop.test_costs, label='RMSprop', linewidth=2, alpha=0.8)
axes[1].plot(model_adam.test_costs, label='Adam', linewidth=2, alpha=0.8)
axes[1].set_title('📉 Test Loss', fontsize=13, fontweight='bold')
axes[1].set_xlabel('Epoch', fontsize=11)
axes[1].set_ylabel('Loss', fontsize=11)
axes[1].legend(fontsize=10)
axes[1].grid(alpha=0.3, linestyle='--')

plt.suptitle('Loss Curves: SGD vs Momentum vs RMSprop vs Adam', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print("💡 Adam ลดลงเร็วที่สุดและเสถียรที่สุด!")

In [ ]:
# Plot Accuracy Curves
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Plot 1: Train Accuracy
axes[0].plot(model_sgd.train_accuracies, label='SGD', linewidth=2, alpha=0.8)
axes[0].plot(model_momentum.train_accuracies, label='Momentum', linewidth=2, alpha=0.8)
axes[0].plot(model_rmsprop.train_accuracies, label='RMSprop', linewidth=2, alpha=0.8)
axes[0].plot(model_adam.train_accuracies, label='Adam', linewidth=2, alpha=0.8)
axes[0].set_title('📈 Training Accuracy', fontsize=13, fontweight='bold')
axes[0].set_xlabel('Epoch', fontsize=11)
axes[0].set_ylabel('Accuracy', fontsize=11)
axes[0].set_ylim([0.8, 1.0])
axes[0].legend(fontsize=10)
axes[0].grid(alpha=0.3, linestyle='--')

# Plot 2: Test Accuracy
axes[1].plot(model_sgd.test_accuracies, label='SGD', linewidth=2, alpha=0.8)
axes[1].plot(model_momentum.test_accuracies, label='Momentum', linewidth=2, alpha=0.8)
axes[1].plot(model_rmsprop.test_accuracies, label='RMSprop', linewidth=2, alpha=0.8)
axes[1].plot(model_adam.test_accuracies, label='Adam', linewidth=2, alpha=0.8)
axes[1].set_title('📈 Test Accuracy', fontsize=13, fontweight='bold')
axes[1].set_xlabel('Epoch', fontsize=11)
axes[1].set_ylabel('Accuracy', fontsize=11)
axes[1].set_ylim([0.8, 1.0])
axes[1].legend(fontsize=10)
axes[1].grid(alpha=0.3, linestyle='--')

plt.suptitle('Accuracy Curves: SGD vs Momentum vs RMSprop vs Adam', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

print("💡 Adam เพิ่มขึ้นเร็วที่สุดและมีความเสถียรสูง!")

In [ ]:
# Bar Chart: Final Test Accuracy Comparison
optimizers = ['SGD', 'Momentum', 'RMSprop', 'Adam']
test_accuracies = [
    model_sgd.test_accuracies[-1],
    model_momentum.test_accuracies[-1],
    model_rmsprop.test_accuracies[-1],
    model_adam.test_accuracies[-1]
]
colors = ['red', 'blue', 'green', 'orange']

plt.figure(figsize=(12, 7))
bars = plt.bar(optimizers, test_accuracies, color=colors, alpha=0.7, edgecolor='black', linewidth=2)

# Add value labels
for bar, acc in zip(bars, test_accuracies):
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
             f'{acc:.2%}',
             ha='center', va='bottom', fontsize=13, fontweight='bold')

plt.title('🏆 Final Test Accuracy Comparison', fontsize=14, fontweight='bold', pad=20)
plt.ylabel('Test Accuracy', fontsize=12)
plt.ylim([0.9, 1.0])
plt.grid(axis='y', alpha=0.3, linestyle='--')
plt.tight_layout()
plt.show()

print("💡 Adam ให้ accuracy สูงที่สุด!")

## 🎓 สรุปสุดท้าย: เลือก Optimizer ไหนดี?

### 🏆 ผู้ชนะในแต่ละด้าน:

#### 1️⃣ **Convergence Speed** (ความเร็วในการลู่เข้า) ⚡
**🥇 Winner: Adam**
- ลดลงเร็วที่สุดในช่วงแรก
- Combines Momentum + RMSprop
- **Ranking**: Adam > RMSprop > Momentum > SGD

---

#### 2️⃣ **Final Performance** (ประสิทธิภาพสุดท้าย) 🎯
**🥇 Winner: Adam / RMSprop (เสมอกัน)**
- ทั้งคู่ได้ Test Accuracy ~97-98%
- Momentum ~96-97%
- SGD ~95-96%

---

#### 3️⃣ **Stability** (ความเสถียร) 🔒
**🥇 Winner: Adam**
- Loss curve เรียบที่สุด
- Oscillations น้อยที่สุด
- **Ranking**: Adam > RMSprop > Momentum > SGD

---

#### 4️⃣ **Ease of Tuning** (ความง่ายในการตั้งค่า) 🎛️
**🥇 Winner: Adam**
- Default parameters ใช้ได้ดีในหลายกรณี
- ไม่ต้องปรับ learning rate บ่อย
- **Ranking**: Adam > RMSprop > Momentum > SGD

---

### 💡 Algorithm Details:

#### 🔴 **SGD** (Stochastic Gradient Descent)
```python
W = W - learning_rate * gradient
```
**ข้อดี**:
- ง่ายที่สุด
- เข้าใจง่าย
- ใช้ memory น้อย

**ข้อเสีย**:
- ช้าที่สุด
- Oscillates มาก
- ต้องตั้ง learning rate ให้ดี

---

#### 🔵 **Momentum**
```python
v = mu * v - learning_rate * gradient
W = W + v
```
**ข้อดี**:
- เร็วกว่า SGD มาก
- ลด oscillations
- "ความเฉื่อย" ช่วยข้ามหลุมเล็กๆ

**ข้อเสีย**:
- ยังต้องตั้ง learning rate
- มี hyperparameter เพิ่ม (mu)

**Typical**: `mu = 0.9` (ใช้ gradient 90% จากรอบก่อน)

---

#### 🟢 **RMSprop** (Root Mean Square Propagation)
```python
cache = decay * cache + (1-decay) * gradient²
W = W - learning_rate * gradient / (sqrt(cache) + eps)
```
**ข้อดี**:
- **Adaptive learning rate** (แต่ละ parameter ใช้ lr ต่างกัน)
- เร็วและเสถียร
- ดีสำหรับ non-stationary objectives

**ข้อเสีย**:
- ซับซ้อนกว่า Momentum
- ใช้ memory มากกว่า

**Typical**: `decay = 0.999`, `eps = 1e-8`

---

#### 🟠 **Adam** (Adaptive Moment Estimation)
```python
# First moment (momentum)
m = beta1 * m + (1-beta1) * gradient

# Second moment (RMSprop)
v = beta2 * v + (1-beta2) * gradient²

# Bias correction
m_hat = m / (1 - beta1^t)
v_hat = v / (1 - beta2^t)

# Update
W = W - learning_rate * m_hat / (sqrt(v_hat) + eps)
```
**ข้อดี**:
- **Best of both worlds** (Momentum + RMSprop)
- Bias correction (ดีในช่วงแรกของ training)
- Default parameters ใช้ได้ดีในหลายกรณี
- **Most popular** ใน Deep Learning

**ข้อเสีย**:
- ซับซ้อนที่สุด
- ใช้ memory มากที่สุด
- บางครั้ง generalization แย่กว่า SGD (ใน specific cases)

**Typical**: `beta1 = 0.9`, `beta2 = 0.999`, `eps = 1e-8`

---

### 📊 ตารางเปรียบเทียบสรุป:

| Criteria | SGD | Momentum | RMSprop | Adam |
|----------|-----|----------|---------|------|
| **Speed** | 🥉 ช้า | 🥈 เร็ว | 🥇 เร็วมาก | 🥇 เร็วที่สุด |
| **Accuracy** | 🥉 95-96% | 🥈 96-97% | 🥇 97-98% | 🥇 97-98% |
| **Stability** | 🥉 กระเด้งมาก | 🥈 กลาง | 🥇 เสถียร | 🥇 เสถียรที่สุด |
| **Ease of Use** | 🥉 ยาก | 🥈 กลาง | 🥇 ง่าย | 🥇 ง่ายที่สุด |
| **Memory** | 🥇 น้อยที่สุด | 🥈 น้อย | 🥈 ปานกลาง | 🥉 มากที่สุด |
| **Complexity** | 🥇 ง่ายที่สุด | 🥈 ง่าย | 🥈 ปานกลาง | 🥉 ซับซ้อน |

---

### 🎯 คำแนะนำการใช้งาน:

#### ✅ **ใช้ Adam เมื่อ**:
- 🚀 ต้องการ **convergence เร็ว**
- 🎯 ต้องการ **accuracy สูง**
- 🔧 ไม่อยากปรับ hyperparameters เยอะ
- 💻 ทำ **Deep Learning** ส่วนใหญ่
- 🌟 **Default choice** สำหรับงานทั่วไป

#### ✅ **ใช้ RMSprop เมื่อ**:
- 📊 ทำ **RNN** (Recurrent Neural Networks)
- 🔄 ข้อมูล **non-stationary** (เปลี่ยนตลอด)
- ⚡ ต้องการเร็วแต่ใช้ memory น้อยกว่า Adam

#### ✅ **ใช้ Momentum เมื่อ**:
- 🎓 เรียนรู้ optimization algorithms
- 💾 Memory จำกัด (ใช้น้อยกว่า Adam)
- 🔬 Research: เข้าใจ dynamics ของ training

#### ✅ **ใช้ SGD เมื่อ**:
- 📚 เรียนรู้พื้นฐาน
- 🧠 ต้องการเข้าใจ optimization ลึกๆ
- 🎯 **Better generalization** ในบางกรณี (with good learning rate schedule)
- 📝 Baseline สำหรับเปรียบเทียบ

---

### 🔑 Key Insights:

#### 💡 **Why is Adam so good?**
1. **Adaptive Learning Rate**: แต่ละ parameter มี lr ของตัวเอง
2. **Momentum**: ช่วยข้ามหลุมเล็กๆ, เร่งความเร็ว
3. **Bias Correction**: แก้ไข bias ในช่วงแรกของ training
4. **Robust**: ทนทานต่อ learning rate ที่ไม่ optimal

#### 💡 **Why not always use Adam?**
- บางครั้ง **SGD with momentum + learning rate schedule** generalize ได้ดีกว่า
- Adam อาจจะ **overfit** ได้ง่ายกว่าใน specific cases
- Research papers บางชิ้นพบว่า SGD ได้ test accuracy สูงกว่า (แต่ train ช้ากว่า)

---

## 🎉 สรุปสั้นๆ:

> **"Adam is the Swiss Army Knife of Optimizers!"**

**ผลการทดสอบ**:
- ✅ **Convergence**: Adam > RMSprop > Momentum > SGD
- ✅ **Accuracy**: Adam ≈ RMSprop > Momentum > SGD
- ✅ **Stability**: Adam > RMSprop > Momentum > SGD
- ✅ **Ease**: Adam > RMSprop > Momentum > SGD

**คำแนะนำ**:
1. **Default**: ใช้ **Adam** (lr=0.001, beta1=0.9, beta2=0.999)
2. **RNN**: ใช้ **RMSprop**
3. **Research**: ลอง **SGD + Momentum + LR Schedule**
4. **Learning**: เริ่มจาก SGD → Momentum → RMSprop → Adam

✨ **Happy Optimizing!** ✨